In [2]:
import os
import openai
import pandas as pd
import scipy
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from openai.embeddings_utils import cosine_similarity

# Load environment variables (contains OPENAI_API_BASE and OPENAI_API_KEY)
load_dotenv(override=True)

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE') # looks like https://********.openai.azure.com/
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize embedding model
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002', deployment = 'text-embedding-ada-002',  chunk_size=1)

# Embeddings vector explanation

In [3]:
text = "King"

def printEmbeddings(iText: str):
    # Embed a single word
    e = embeddings.embed_query(text)
    #print(e)
    print(f'{iText} -> lunghezza stringa: {len(iText)} - lunghezza vettore: {len(e)}')

printEmbeddings("King")
printEmbeddings("man")
printEmbeddings("woman")

King -> lunghezza stringa: 4 - lunghezza vettore: 1536
man -> lunghezza stringa: 3 - lunghezza vettore: 1536
woman -> lunghezza stringa: 5 - lunghezza vettore: 1536


In [4]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://openai-test-model.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  engine="davinci",
  prompt="Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment\n\nHeadline 1: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet's most beloved cooking guru has a buzzy new book and a fresh new perspective\nCategory: Entertainment\n\nHeadline 2: Major Retailer Announces Plans to Close Over 100 Stores\nCategory:",
  temperature=0,
  max_tokens=60,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

# Embeddings with words

In [5]:
def get_similarity_from_text(text1, text2, varNameText1: str = "", varNameText2: str = "", printText: bool = True):
    e1 = embeddings.embed_query(text1)
    e2 = embeddings.embed_query(text2)
    s = cosine_similarity(e1, e2)

    print(f"Somiglianza tra '{text1 if printText else varNameText1}' e '{text2 if printText else varNameText2}': {s}")


get_similarity_from_text("ragazzo", "cane")
get_similarity_from_text("ragazzo", "bambino")
get_similarity_from_text("ragazzo", "ragazza")


Somiglianza tra 'ragazzo' e 'cane': 0.8019525443639803
Somiglianza tra 'ragazzo' e 'bambino': 0.8620923331020662
Somiglianza tra 'ragazzo' e 'ragazza': 0.9371455305554139


# Python libraries example 

In [6]:
import pandas as pd

df = pd.read_csv('data/libraries_dataset.csv', header=0, sep=';')

print('')

# only keep library_name and description column
df = df[['library_name', 'description']]

# add embedding column with embedding
df['embedding'] = df['description'].apply(lambda x: embeddings.embed_query(x))

print('DataFrame popolato')


DataFrame popolato


# Python libraries example 

In [7]:
# Let's make a search on libraries df based on a particular need
request = "I need a python library to parse html code retrived with an HTTP request"

# get embedding for request
request_embedding = embeddings.embed_query(request)

# get cosine similarity between request and all libraries descriptions
similarities = df['embedding'].apply(lambda x: cosine_similarity(x, request_embedding))

# combine library_name from df and similiaries and sort ascending by similarity
recommendations = pd.concat([df['library_name'], similarities], axis=1).sort_values(by='embedding', ascending=False)
recommendations.head(10)

,library_name,embedding
9,BeautifulSoup,0.819023
8,Scrapy,0.817715
6,Scikit-learn,0.761493
3,Pandas,0.758129
7,PyTorch,0.755811
0,TensorFlow,0.755700
1,SciPy,0.746985
4,Matplotlib,0.724728
5,Keras,0.711553
2,Numpy,0.703839


# Confronto diagnosi

In [8]:
case1 = """Paziente: Maschio, 45 anni
Descrizione dei sintomi: Febbre persistente, affaticamento, tosse secca, perdita di peso involontaria e sudorazione notturna.
Diagnosi: Dopo un'attenta valutazione dei sintomi riportati dal paziente, esami fisici e risultati di test di laboratorio, la diagnosi principale è sospetta tubercolosi polmonare.
Motivazione della diagnosi: La febbre persistente, l'affaticamento, la tosse secca e la sudorazione notturna sono sintomi tipici della tubercolosi polmonare. La perdita di peso involontaria è un segno comune nelle fasi avanzate della malattia. La tubercolosi polmonare è una condizione infettiva causata dal batterio Mycobacterium tuberculosis.
Ulteriori indagini: Saranno necessari ulteriori esami per confermare la diagnosi. Si raccomanda una radiografia del torace, una coltura del muco bronchiale e un test cutaneo alla tubercolina per valutare la presenza dell'infezione.
Piano di trattamento: Il paziente dovrà iniziare immediatamente una terapia antibiotica specifica per la tubercolosi polmonare. Sarà necessario un trattamento di lunga durata, generalmente composto da una combinazione di farmaci antibatterici. Il paziente sarà monitorato regolarmente per valutare la risposta al trattamento e gestire eventuali effetti collaterali dei farmaci.
Follow-up: Il paziente dovrà essere sottoposto a controlli periodici per monitorare l'efficacia del trattamento e l'eventuale comparsa di complicanze. Sarà importante mantenere uno stretto contatto con il medico curante e seguire attentamente le istruzioni riguardanti l'assunzione dei farmaci e il controllo degli effetti collaterali."""

case2 = """Paziente: Femmina, 35 anni
Descrizione dei sintomi: Dolore addominale acuto, localizzato nella parte inferiore destra dell'addome, nausea e lieve febbre.
Diagnosi: Dopo una valutazione dei sintomi e un esame fisico, la diagnosi principale è una possibile appendicite acuta.
Motivazione della diagnosi: Il dolore addominale acuto, localizzato nella parte inferiore destra, è un sintomo tipico dell'appendicite acuta. La nausea e la lieve febbre possono essere associati a un'infiammazione dell'appendice.
Ulteriori indagini: Saranno necessari ulteriori esami per confermare la diagnosi. Si consiglia una serie di esami del sangue per valutare i livelli di globuli bianchi e una ecografia addominale per valutare la presenza di infiammazione dell'appendice.
Piano di trattamento: Il paziente dovrà essere sottoposto a un intervento chirurgico d'urgenza per rimuovere l'appendice infiammata (appendicectomia). Questa procedura viene generalmente eseguita laparoscopicamente, consentendo una rapida ripresa postoperatoria.
Follow-up: Dopo l'intervento chirurgico, il paziente dovrà seguire le istruzioni del medico per la gestione del dolore, la cura delle ferite e la ripresa delle normali attività. Saranno programmati appuntamenti di follow-up per valutare la guarigione e risolvere eventuali complicanze postoperatorie."""

case3 = """Paziente: Maschio, 45 anni
Descrizione dei sintomi: Dolore persistente all'addome superiore, affaticamento, perdita di appetito, lieve nausea e senso di gonfiore addominale.
Diagnosi: Dopo una valutazione dei sintomi riportati dalla paziente, esami fisici e risultati di test di laboratorio, la diagnosi principale è sospetta gastrite cronica.
Motivazione della diagnosi: Il dolore persistente all'addome superiore, la perdita di appetito, la lieve nausea e il senso di gonfiore addominale sono sintomi comuni della gastrite cronica. Questa condizione è caratterizzata da un'infiammazione della mucosa dello stomaco.
Ulteriori indagini: Saranno necessari ulteriori esami per confermare la diagnosi. Si raccomanda una gastroscopia per visualizzare direttamente la mucosa gastrica e prelevare campioni per l'analisi dei tessuti.
Piano di trattamento: Il paziente dovrà seguire una dieta bland-a e evitare cibi piccanti, alcolici e caffeina. Sarà prescritta una terapia farmacologica per ridurre l'infiammazione dello stomaco e alleviare i sintomi. In alcuni casi, potrebbe essere necessario un trattamento a lungo termine.

Follow-up: Il paziente dovrà effettuare controlli periodici per monitorare la risposta al trattamento e apportare eventuali modifiche alla terapia. Sarà importante seguire attentamente le indicazioni del medico curante e adottare uno stile di vita sano, evitando fattori che possono aggravare la gastrite cronica."""


get_similarity_from_text(case1, case2, "case1", "case2", False)
get_similarity_from_text(case2, case3, "case2", "case3", False)

Somiglianza tra 'case1' e 'case2': 0.8302775799441025
Somiglianza tra 'case2' e 'case3': 0.9059029085686701
